In [1]:
import pandas as pd
import joblib

# Load the trained V2-B pipeline
pipeline = joblib.load("models/siem_lr_v2b_pipeline.joblib")

print("Pipeline loaded successfully")


Pipeline loaded successfully


In [2]:
# Example incoming SIEM event (single record)
incoming_event = {
    "dst_port": 22,
    "failed_attempts": 6,
    "session_duration": 45,
    "bytes_sent": 1200,
    "event_type": "auth",
    "src_ip_type": "external",
    "protocol": "tcp",
    "alert_severity": "high",
    "is_cloud_asset": 1
}

incoming_event


{'dst_port': 22,
 'failed_attempts': 6,
 'session_duration': 45,
 'bytes_sent': 1200,
 'event_type': 'auth',
 'src_ip_type': 'external',
 'protocol': 'tcp',
 'alert_severity': 'high',
 'is_cloud_asset': 1}

In [3]:
# Convert incoming event to DataFrame (model expects tabular input)
event_df = pd.DataFrame([incoming_event])

event_df


,dst_port,failed_attempts,session_duration,bytes_sent,event_type,src_ip_type,protocol,alert_severity,is_cloud_asset
0,22,6,45,1200,auth,external,tcp,high,1


In [4]:
# Generate attack probability
attack_probability = pipeline.predict_proba(event_df)[0][1]

attack_probability


np.float64(0.9999596059361664)

In [5]:
# Configurable SOC threshold
THRESHOLD = 0.7

# Final decision
prediction = "ATTACK" if attack_probability >= THRESHOLD else "BENIGN"

prediction


'ATTACK'

In [6]:
# Final production inference output
output = {
    "attack_probability": round(float(attack_probability), 6),
    "threshold": THRESHOLD,
    "prediction": prediction,
    "model_name": "logistic_regression",
    "baseline": "V2-B",
    "signature_free": True
}

output


{'attack_probability': 0.99996,
 'threshold': 0.7,
 'prediction': 'ATTACK',
 'model_name': 'logistic_regression',
 'baseline': 'V2-B',
 'signature_free': True}